In [66]:
import stim

In [97]:
circuit = stim.Circuit('''
    R 0 1 2 3 4
    TICK
    CX 0 1 2 3
    TICK
    CX 2 1 4 3
    TICK
    MR 1 3
    DETECTOR(1, 0) rec[-2]
    DETECTOR(3, 0) rec[-1]
    TICK
    CX 0 1 2 3
    TICK
    CX 2 1 4 3
    TICK
    MR 1 3
    SHIFT_COORDS(0, 1)
    DETECTOR(1, 0) rec[-2] rec[-4]
    DETECTOR(3, 0) rec[-1] rec[-3]
    M 0 2 4
    DETECTOR(1, 1) rec[-2] rec[-3] rec[-5]
    DETECTOR(3, 1) rec[-1] rec[-2] rec[-4]
    OBSERVABLE_INCLUDE(0) rec[-1]
''')

In [98]:
circuit.flattened()

stim.Circuit('''
    R 0 1 2 3 4
    TICK
    CX 0 1 2 3
    TICK
    CX 2 1 4 3
    TICK
    MR 1 3
    DETECTOR(1, 0) rec[-2]
    DETECTOR(3, 0) rec[-1]
    TICK
    CX 0 1 2 3
    TICK
    CX 2 1 4 3
    TICK
    MR 1 3
    DETECTOR(1, 1) rec[-2] rec[-4]
    DETECTOR(3, 1) rec[-1] rec[-3]
    M 0 2 4
    DETECTOR(1, 2) rec[-2] rec[-3] rec[-5]
    DETECTOR(3, 2) rec[-1] rec[-2] rec[-4]
    OBSERVABLE_INCLUDE(0) rec[-1]
''')

In [87]:
# the keys of the dictionary must match the qubit labels of the circuit
# the units of T1, T2 and operation durations must be the same, in this case microseconds. 

t1 = {0: 55.4, 1: 50.1, 2: 56.9, 3: 51.4, 4: 52.0}
t2 = {0: 22.3, 1: 20.6, 2: 26.1, 3: 23.9, 4: 21.7}

In [88]:
# all the operations in the circuit should be included here

op_duration = {"MR": 153, "M": 51, "CX": 40, "R": 102}

In [95]:
from typing import Tuple

import numpy as np
import stim
from pymatching import Matching

MEASUREMENTS = ["M", "MPP", "MR", "MRX", "MRY", "MRZ", "MX", "MY", "MZ"]
ANNOTATIONS = ["QUBIT_COORDS", "DETECTOR", "OBSERVABLE_INCLUDE", "TICK", "SHIFT_COORDS"]


def add_t1t2_noise(
    circuit: stim.Circuit,
    t1: dict,
    t2: dict,
    op_duration: dict,
    symmetrize_noise: bool = True,
) -> stim.Circuit:
    """
    Returns circuit with amplitude and phase damping noise
    using the Pauli Twirl Approximation (PTA).

    Paramaters
    ----------
    circuit
        Stim circuit containing the QEC experiment without noise.
        It must have TICKs separating blocks of operations
        that are executed in parallel.
        Note: this function assumes that all qubits are measured
        at some point in the circuit.
    t1
        Dictionary with the T1 times of every qubit in the circuit
        following the same labelling as in the stim circuit.
        Note: use same units for t1, t2 and op_duration.
    t2
        Dictionary with the T2 times of every qubit in the circuit
        following the same labelling as in the stim circuit.
        Note: use same units for t1, t2 and op_duration.
    op_duration
        Dictionary with the operation times of every operation in the circuit
        following the same labelling as in the stim circuit.
        Note: use same units for t1, t2 and op_duration.
    symmetrize_noise
        Bool to add the noise symmetrically before and after the gate.

    Returns
    -------
    noisy_circuit
        Stim circuit corresponding to 'circuit' with the amplitude and phase
        damping noise under the PTA.
        Note: returns the circuit flattened.
    """
    # checks
    for q in t1:
        if t2[q] > 2 * t1[q]:
            raise ValueError(f"T2 > 2 * T1 for qubit {q}")

    if circuit.flattened() != circuit.without_noise().flattened():
        raise ValueError("The circuit provided contains noise")
    if circuit.num_ticks == 0:
        raise ValueError("The circuit must contain at least one TICK")

    qubits = [i.targets_copy() for i in circuit.flattened() if i.name in MEASUREMENTS]
    qubits = [x for xs in qubits for x in xs]  # flatten list
    qubits = set([q.qubit_value for q in qubits])
    if qubits - set(t1):
        raise ValueError(
            "T1 must contain all the qubits in the circuit,"
            f"missing the following qubits: {set(qubits) - set(t1)}"
        )
    if qubits - set(t2):
        raise ValueError(
            "T2 must contain all the qubits in the circuit,"
            f"missing the following qubits: {set(qubits) - set(t2)}"
        )

    # prepare inputs
    duration = {i: 0 for i in ANNOTATIONS}
    duration.update(op_duration)
    # adds TICK at the end of the circuit to include last block
    # note: make a copy of the circuit (using flattened)
    circuit_flattened = circuit.flattened()
    circuit_flattened.append(stim.CircuitInstruction("TICK", [], []))

    # add noise
    noisy_circuit = stim.Circuit()
    block = []
    length = len(circuit_flattened)

    for k, instr in enumerate(circuit_flattened):
        print(f"Instruction {k}: ", instr.name)
        print(f"Cur block: ", block)
        if instr.name not in duration:
            raise ValueError(f"{instr.name} not in 'op_duration'")

        if instr.name != "TICK":
            block.append(instr)
            continue

        # add noise in block
        t = max([duration[i.name] for i in block], default=0)
        print(t)

        if symmetrize_noise:
            block = add_symmetric_noise_to_block(block, t, t1, t2, qubits)
        else:
            block = add_noise_to_block(block, t, t1, t2, qubits)

        # append block to noisy circuit
        for instr in block:
            noisy_circuit.append(instr)
        if k != length - 1:
            # does not add last TICK that was artificially added
            noisy_circuit.append(stim.CircuitInstruction("TICK", [], []))

        block = []

    return noisy_circuit


def add_noise_to_block(block: list, t: float, t1: dict, t2: dict, qubits: list):
    """
    Add noise to the specified qubits at the end of the block given t, T1, T2.

    Parameters
    ----------
    block
        List of operations to the qubits
    t
        Duration of the block.
        Note: use same units for t1, t2 and t.
    t1
        Dictionary with the T1 times of every qubit in the circuit
        following the same labelling as in the stim circuit.
        Note: use same units for t1, t2 and t.
    t2
        Dictionary with the T2 times of every qubit in the circuit
        following the same labelling as in the stim circuit.
        Note: use same units for t1, t2 and t.
    qubits
        Qubits in which to add noise

    Returns
    -------
    block
        Same list of operations with noise
    """
    for q in qubits:
        px, py, pz = get_perror_from_t1t2(t=t, t1=t1[q], t2=t2[q])
        block.append(
            stim.CircuitInstruction(
                name="PAULI_CHANNEL_1", targets=[q], gate_args=[px, py, pz]
            )
        )
    return block


def add_symmetric_noise_to_block(
    block: list, t: float, t1: dict, t2: dict, qubits: list
):
    """
    Add symmetric noise to the specified qubits at the beginning
    and end of the block given t, T1, T2.

    Parameters
    ----------
    block
        List of operations to the qubits
    t
        Duration of the block.
        Note: use same units for t1, t2 and t.
    t1
        Dictionary with the T1 times of every qubit in the circuit
        following the same labelling as in the stim circuit.
        Note: use same units for t1, t2 and t.
    t2
        Dictionary with the T2 times of every qubit in the circuit
        following the same labelling as in the stim circuit.
        Note: use same units for t1, t2 and t.
    qubits
        Qubits in which to add noise

    Returns
    -------
    block
        Same list of operations with noise
    """
    for q in qubits:
        px, py, pz = get_perror_from_t1t2(t=t / 2, t1=t1[q], t2=t2[q])
        noise_circ = stim.CircuitInstruction(
            name="PAULI_CHANNEL_1", targets=[q], gate_args=[px, py, pz]
        )
        block = [noise_circ, *block, noise_circ]
    return block


def get_perror_from_t1t2(t: float, t1: float, t2: float) -> Tuple[float, float, float]:
    """
    Returns the single-qubit Pauli error probabilities for the
    given duration t and T1 and T2 times, from an amplitude and
    phase damping noise with the Pauli Twirl Approximation.

    References:
    arXiv:1210.5799v2
    arXiv:1305.2021v1

    Paramters
    ---------
    t:
        Duration of the noise
    t1:
        T1 time
    t2:
        T2 time

    Returns
    -------
    px:
        X error probability
    py:
        Y error probability
    pz:
        Z error probability
    """
    if t == 0:
        return 0, 0, 0

    px = 0.25 * (1 - np.exp(-t / t1))
    py = px
    pz = 0.5 * (1 - np.exp(-t / t2)) - 0.25 * (1 - np.exp(-t / t1))
    return px, py, pz


def get_mwpm(
    circuit: stim.Circuit,
    t1: dict,
    t2: dict,
    op_duration: dict,
    approximate_disjoint_errors: bool = False,
    symmetrize_noise: bool = True,
) -> Matching:
    """
    Returns a MWPM initialized with amplitude and phase damping noise
    using the Pauli Twirl Approximation (PTA).

    Paramaters
    ----------
    circuit
        Stim circuit containing the QEC experiment without noise.
        It must have TICKs separating blocks of operations
        that are executed in parallel.
        Note: this function assumes that all qubits are measured
        at some point in the circuit.
    t1
        Dictionary with the T1 times of every qubit in the circuit
        following the same labelling as in the stim circuit.
        Note: use same units for t1, t2 and op_duration.
    t2
        Dictionary with the T2 times of every qubit in the circuit
        following the same labelling as in the stim circuit.
        Note: use same units for t1, t2 and op_duration.
    op_duration
        Dictionary with the operation times of every operation in the circuit
        following the same labelling as in the stim circuit.
        Note: use same units for t1, t2 and op_duration.
    approximate_disjoint_errors
        Flag for stim.Circuit.detector_error_model().
        See stim's documentation.
    symmetrize_noise
        Bool to add the noise symmetrically before and after the gate.

    Returns
    -------
    mwpm
        pymatching.Matching initialized with amplitude and phase damping noise
        using the Pauli Twirl Approximation (PTA) for the given circuit.
    """

    noisy_circuit = add_t1t2_noise(
        circuit=circuit,
        t1=t1,
        t2=t2,
        op_duration=op_duration,
        symmetrize_noise=symmetrize_noise,
    )
    dem = noisy_circuit.detector_error_model(
        decompose_errors=True,
        allow_gauge_detectors=True,
        approximate_disjoint_errors=approximate_disjoint_errors,
    )
    mwpm = Matching(dem)

    return mwpm


In [96]:
noisy_circuit = add_t1t2_noise(
        circuit=circuit,
        t1=t1,
        t2=t2,
        op_duration=op_duration,
        symmetrize_noise=False,
    )

Instruction 0:  R
Cur block:  []
Instruction 1:  TICK
Cur block:  [stim.CircuitInstruction('R', [stim.GateTarget(0), stim.GateTarget(1), stim.GateTarget(2), stim.GateTarget(3), stim.GateTarget(4)], [])]
102
Instruction 2:  CX
Cur block:  []
Instruction 3:  TICK
Cur block:  [stim.CircuitInstruction('CX', [stim.GateTarget(0), stim.GateTarget(1), stim.GateTarget(2), stim.GateTarget(3)], [])]
40
Instruction 4:  CX
Cur block:  []
Instruction 5:  TICK
Cur block:  [stim.CircuitInstruction('CX', [stim.GateTarget(2), stim.GateTarget(1), stim.GateTarget(4), stim.GateTarget(3)], [])]
40
Instruction 6:  MR
Cur block:  []
Instruction 7:  DETECTOR
Cur block:  [stim.CircuitInstruction('MR', [stim.GateTarget(1), stim.GateTarget(3)], [])]
Instruction 8:  DETECTOR
Cur block:  [stim.CircuitInstruction('MR', [stim.GateTarget(1), stim.GateTarget(3)], []), stim.CircuitInstruction('DETECTOR', [stim.GateTarget(stim.target_rec(-2))], [1, 0])]
Instruction 9:  TICK
Cur block:  [stim.CircuitInstruction('MR', [sti

In [91]:
noisy_circuit

stim.Circuit('''
    R 0 1 2 3 4
    PAULI_CHANNEL_1(0.210341, 0.210341, 0.2845) 0
    PAULI_CHANNEL_1(0.21736, 0.21736, 0.279103) 1
    PAULI_CHANNEL_1(0.208369, 0.208369, 0.281591) 2
    PAULI_CHANNEL_1(0.215635, 0.215635, 0.277358) 3
    PAULI_CHANNEL_1(0.21484, 0.21484, 0.280615) 4
    TICK
    CX 0 1 2 3
    PAULI_CHANNEL_1(0.128558, 0.128558, 0.288272) 0
    PAULI_CHANNEL_1(0.137488, 0.137488, 0.290785) 1
    PAULI_CHANNEL_1(0.126224, 0.126224, 0.265786) 2
    PAULI_CHANNEL_1(0.135193, 0.135193, 0.271025) 3
    PAULI_CHANNEL_1(0.134158, 0.134158, 0.286697) 4
    TICK
    CX 2 1 4 3
    PAULI_CHANNEL_1(0.128558, 0.128558, 0.288272) 0
    PAULI_CHANNEL_1(0.137488, 0.137488, 0.290785) 1
    PAULI_CHANNEL_1(0.126224, 0.126224, 0.265786) 2
    PAULI_CHANNEL_1(0.135193, 0.135193, 0.271025) 3
    PAULI_CHANNEL_1(0.134158, 0.134158, 0.286697) 4
    TICK
    MR 1 3
    DETECTOR(1, 0) rec[-2]
    DETECTOR(3, 0) rec[-1]
    PAULI_CHANNEL_1(0.234204, 0.234204, 0.265272) 0
    PAULI_CHANNEL_1

In [73]:
duration

{'QUBIT_COORDS': 0,
 'DETECTOR': 0,
 'OBSERVABLE_INCLUDE': 0,
 'TICK': 0,
 'SHIFT_COORDS': 0,
 'MR': 153,
 'M': 51,
 'CX': 40,
 'R': 102}

In [74]:
qubits

{0, 1, 2, 3, 4}

In [75]:
circuit_flattened

stim.Circuit('''
    R 0 1 2 3 4
    TICK
    CX 0 1 2 3
    TICK
    CX 2 1 4 3
    TICK
    MR 1 3
    DETECTOR(1, 0) rec[-2]
    DETECTOR(3, 0) rec[-1]
    TICK
    CX 0 1 2 3
    TICK
    CX 2 1 4 3
    TICK
    MR 1 3
    DETECTOR(1, 1) rec[-2] rec[-4]
    DETECTOR(3, 1) rec[-1] rec[-3]
    M 0 2 4
    DETECTOR(1, 2) rec[-2] rec[-3] rec[-5]
    DETECTOR(3, 2) rec[-1] rec[-2] rec[-4]
    OBSERVABLE_INCLUDE(0) rec[-1]
    TICK
''')

In [76]:
from typing import Tuple
from warnings import warn
from math import exp, inf

def idle_error_probs(
    relax_time: float, deph_time: float, duration: float
) -> Tuple[float]:
    """
    idle_error_probs Returns the probabilities of X, Y, and Z errors
    for a Pauli-twirled amplitude and phase damping channel.

    Parameters
    ----------
    relax_time : float
        The relaxation time (T_1) of the qubit
    deph_time : float
        The dephasing time (T_2) of the qubit
    duration : float
        The duration of the amplitude-phase damping period

    Returns
    -------
    Tuple[float]
        The probabilities of X, Y, and Z errors
    """
    if relax_time <= 0:
        raise ValueError("The relaxation time ('relax_time') must be positive")
    if deph_time <= 0:
        raise ValueError("The dephasing time ('deph_time') must be positive")
    if duration <= 0: 
        raise ValueError("The idling duration ('duration') must be positive")

    relax_prob = 1 - exp(-duration / relax_time)
    deph_prob = 1 - exp(-duration / deph_time)

    x_prob = y_prob = 0.25 * relax_prob
    z_prob = 0.5 * deph_prob - 0.25 * relax_prob

    return x_prob, y_prob, z_prob


In [77]:
from stim import CircuitInstruction
from typing import Iterator, Sequence

from itertools import repeat

relax_times = list(repeat(30000, 5))
deph_times = list(repeat(30000, 5))

def idle(qubits: Sequence[int], duration: float) -> Iterator[CircuitInstruction]:
    """
    idle Returns the circuit instructions for an idling period on the given qubits.

    Parameters
    ----------
    qubits : Sequence[str]
        The qubits to idle.
    duration : float
        The duration of the idling period.

    Yields
    ------
    Iterator[CircuitInstruction]
        The circuit instructions for an idling period on the given qubits.
    """
    for qubit in qubits:
        relax_time = relax_times[qubit]
        deph_time = deph_times[qubit]

        error_probs = list(idle_error_probs(relax_time, deph_time, duration))
        yield CircuitInstruction("PAULI_CHANNEL_1", [qubit], error_probs)

In [79]:
for inst in idle((0, 1, 2), 500):
    print(inst)

PAULI_CHANNEL_1(0.00413214, 0.00413214, 0.00413214) 0
PAULI_CHANNEL_1(0.00413214, 0.00413214, 0.00413214) 1
PAULI_CHANNEL_1(0.00413214, 0.00413214, 0.00413214) 2


In [80]:
def gate(qubits: Sequence[int]) -> Iterator[CircuitInstruction]:
    """
    gate Returns the circuit instructions for a gate on the given qubits.

    Parameters
    ----------
    qubits : Sequence[str]
        The qubits to apply the gate to.
    duration : float
        The duration of the gate.
    gate_name : str
        The name of the gate.

    Yields
    ------
    Iterator[CircuitInstruction]
        The circuit instructions for a gate on the given qubits.
    """
    yield CircuitInstruction("X", qubits)
    yield from idle(qubits, 100)

In [82]:
for inst in gate((0, 1, 2)):
    print(inst)

X 0 1 2
PAULI_CHANNEL_1(0.000831946, 0.000831946, 0.000831946) 0
PAULI_CHANNEL_1(0.000831946, 0.000831946, 0.000831946) 1
PAULI_CHANNEL_1(0.000831946, 0.000831946, 0.000831946) 2
